In [7]:
import pandas as pd
import numpy as np

# Assuming you've loaded your datasets
base_model_df = pd.read_csv('full_stats.csv')
sequence_model_df = pd.read_csv('Model_sequence_stats.csv')

# Function to calculate individual-level statistics for Alpha, Lambda, and Gamma
def calculate_individual_stats(df):
    # Initialize dictionaries to store individual-level stats
    alphas, lambdas, gammas = {}, {}, {}
    
    # Loop over each participant
    for i in range(1, 50):  # Assuming participant indices go from 1 to 49
        alpha_key = f'v_Intercept_subj.{i}'
        v_gain_key = f'v_gain_subj.{i}'
        v_loss_key = f'v_loss_subj.{i}'
        gamma_key = f'z_subj.{i}'
        
        # Check if keys exist in the dataframe to handle missing data
        if alpha_key in df['Unnamed: 0'].values and v_gain_key in df['Unnamed: 0'].values and v_loss_key in df['Unnamed: 0'].values:
            alpha = df[df['Unnamed: 0'] == alpha_key]['mean'].values[0]
            v_gain_mean = df[df['Unnamed: 0'] == v_gain_key]['mean'].values[0]
            v_loss_mean = df[df['Unnamed: 0'] == v_loss_key]['mean'].values[0]
            lambda_value = v_loss_mean / v_gain_mean
            alphas[i] = alpha
            lambdas[i] = lambda_value
        
        if gamma_key in df['Unnamed: 0'].values:
            gamma = df[df['Unnamed: 0'] == gamma_key]['mean'].values[0]
            gammas[i] = gamma
    
    return alphas, lambdas, gammas

# Calculate the individual-level statistics for both models
alphas_base, lambdas_base, gammas_base = calculate_individual_stats(base_model_df)
alphas_sequence, lambdas_sequence, gammas_sequence = calculate_individual_stats(sequence_model_df)

# Example: Print the Alpha, Lambda, and Gamma for the first participant in the base model
print(f"Base Model - Participant 1: Alpha = {alphas_base[1]}, Lambda = {lambdas_base[1]}, Gamma = {gammas_base.get(1, np.nan)}")

# You can similarly access and compare these values for all participants and between models as needed.


Base Model - Participant 1: Alpha = 0.1542285012914002, Lambda = 1.1319739711098875, Gamma = 0.4396552170105913


In [12]:
import pandas as pd

# Load the datasets
base_model_df = pd.read_csv('full_stats.csv')
sequence_model_df = pd.read_csv('Model_sequence_stats.csv')

# Initialize a DataFrame to store individual-level statistics for comparison
individual_comparison = pd.DataFrame(columns=['Participant', 'Model', 'Alpha', 'Lambda', 'Gamma'])

# Function to calculate individual stats and append to the DataFrame
def append_individual_stats(df, model_name):
    global individual_comparison
    for subj_index in range(1, 50):  # Assuming 49 participants indexed from 1 to 49
        alpha_key = f'v_Intercept_subj.{subj_index}'
        v_gain_key = f'v_gain_subj.{subj_index}'
        v_loss_key = f'v_loss_subj.{subj_index}'
        z_key = f'z_subj.{subj_index}'

        # Check if the keys exist in the dataframe to avoid KeyError
        if alpha_key in df['Unnamed: 0'].values and v_gain_key in df['Unnamed: 0'].values and v_loss_key in df['Unnamed: 0'].values:
            alpha = df[df['Unnamed: 0'] == alpha_key]['mean'].values[0]
            v_gain = df[df['Unnamed: 0'] == v_gain_key]['mean'].values[0]
            v_loss = df[df['Unnamed: 0'] == v_loss_key]['mean'].values[0]
            lambda_value = v_loss / v_gain

            # 'z' parameter might not change between models so it's taken from the base model
            gamma = base_model_df[base_model_df['Unnamed: 0'] == z_key]['mean'].values[0] if z_key in base_model_df['Unnamed: 0'].values else None

            # Append to the DataFrame
            individual_comparison = individual_comparison.append({
                'Participant': subj_index,
                'Model': model_name,
                'Alpha': alpha,
                'Lambda': lambda_value,
                'Gamma': gamma
            }, ignore_index=True)
        else:
            print(f"Data for Participant {subj_index} in model {model_name} is incomplete.")

# Calculate individual stats for both models and append
append_individual_stats(base_model_df, 'Base Model')
append_individual_stats(sequence_model_df, 'Model with Trial Sequence')

# Save the individual-level comparison to a CSV file
output_path = '/mnt/data/individual_level_comparison.csv'
individual_comparison.to_csv(output_path, index=False)

print("Individual-level comparison saved to", output_path)


AttributeError: 'DataFrame' object has no attribute 'append'